In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

**Loading Dataset**

In [ ]:
train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')


**Looking into training data**

In [ ]:
train.head()


In [ ]:
train.shape

In [ ]:
train.describe()

In [ ]:
train.describe(include=['O'])

In [ ]:
train.isnull().sum()

**Looking into testing data**

In [ ]:
test.shape

In [ ]:
test.head()

In [ ]:
test.info()

In [ ]:
test.isnull().sum()

**Relationship between different features and Survival**

In [ ]:
survived = train[train['Survived'] == 1]
not_survived = train[train['Survived'] == 0]

In [ ]:
print ("Survived: {}({:.1f}%)".format(len(survived), float(len(survived)/len(train)*100.0)))
print ("Not Survived: {}({:.1f}%)".format(len(not_survived), float(len(not_survived)/len(train)*100.0)))
print ("Total:{}".format(len(train)))

**Pclass v/s Survival**

In [ ]:
train.Pclass.value_counts()

In [ ]:
train.groupby('Pclass').Survived.value_counts()

In [ ]:
train[['Pclass', 'Survived']].groupby('Pclass',as_index=False).Survived.mean()

In [ ]:
sns.barplot(x='Pclass', y='Survived', data=train)

**Sex v/s Survival**

In [ ]:
train.Sex.value_counts()

In [ ]:
train.groupby('Sex').Survived.value_counts()

In [ ]:
train[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean()

In [ ]:
sns.barplot(x='Sex', y='Survived', data=train)


**Sex and Pclass v/s Survival**

In [ ]:
tab = pd.crosstab(train['Pclass'], train['Sex'])
print (tab)

In [ ]:
tab.div(tab.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True)
plt.xlabel('Pclass')
plt.ylabel('Percentage')


In [ ]:
sns.catplot(x='Sex', y='Survived', hue='Pclass', height=4, aspect=2, data=train)

**Pclass, Sex and Embarked v/s Survival**

In [ ]:
sns.catplot(x='Pclass', y='Survived', hue='Sex', col='Embarked', data=train)

**Embarked v/s Survival**

In [ ]:
train.Embarked.value_counts()

In [ ]:
train.groupby('Embarked').Survived.value_counts()


In [ ]:
train[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean()

In [ ]:
sns.barplot(x='Embarked', y='Survived', data=train)


**Parch v/s Survival**

In [ ]:
train.Parch.value_counts()

In [ ]:
train.groupby('Parch').Survived.value_counts()



In [ ]:
train[['Parch','Survived']].groupby('Parch',as_index=False).mean()

In [ ]:
sns.barplot(x='Parch', y='Survived', ci=None, data=train)

**SibSp v/s Survived**

In [ ]:
train.SibSp.value_counts()

In [ ]:
train.groupby('SibSp').Survived.value_counts()

In [ ]:
train[['SibSp','Survived']].groupby('SibSp',as_index=False).mean()

In [ ]:
sns.barplot(x='SibSp',y='Survived',data=train)

In [ ]:
train.Age.value_counts()

In [ ]:
train.groupby('SibSp').Survived.value_counts()

In [ ]:
sns.barplot(x='Age',y='Survived',data=train)

In [ ]:
fig = plt.figure(figsize=(15,5))
ax1 = fig.add_subplot(131)
ax2 = fig.add_subplot(132)
ax3 = fig.add_subplot(133)
sns.violinplot(x="Embarked", y="Age", hue="Survived", data=train, split=True, ax=ax1)
sns.violinplot(x="Pclass", y="Age", hue="Survived", data=train, split=True, ax=ax2)
sns.violinplot(x="Sex", y="Age", hue="Survived", data=train, split=True, ax=ax3)

**Feature Extraction**

In [ ]:
titles=set()
for name in train['Name']:
    titles.add(name.split(',')[1].split('.')[0].strip(' '))
print(titles)  

In [ ]:
title_dict={'Mrs':'Mrs','Major':'Other','Master':'Master','Lady':'Other','Mlle':'Miss','Dr':'Other','Col':'Other','Capt':'Other','Don':'Other','the Countess':'Other','Mme':'Mrs','Miss':'Miss','Jonkheer':'Other','Rev':'Other','Sir':'Other','Ms':'Miss','Mr':'Mr'}

In [ ]:
train_test=[train,test]

In [ ]:
for dataset in train_test:
    dataset['Title']=dataset['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip() )
    dataset['Title']=dataset.Title.map(title_dict)  

In [ ]:
train.head()

In [ ]:
train.tail()

In [ ]:
pd.crosstab(train['Title'],train['Sex'])

In [ ]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Other": 5}

In [ ]:
for dataset in train_test:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    
train.head()

In [ ]:
for dataset in train_test:
    train['Title']=train['Title'].fillna(0)

In [ ]:
train.head()

**Cleaning the dataset**

In [ ]:
df1=train.drop(['Name','Ticket','PassengerId','Cabin','Embarked'],axis=1)
df1.head()

In [ ]:
df1['Sex']=df1.Sex.map({'female':1,'male':0})

In [ ]:
df1.head()

In [ ]:
df1.isnull().sum()

In [ ]:
mean_men=df1[df1['Sex']==0]['Age'].mean()
mean_fem=df1[df1['Sex']==1]['Age'].mean()

In [ ]:
df1.loc[(df1.Age.isnull())&(df1['Sex']==1),'Age']=mean_fem
df1.loc[(df1.Age.isnull())&(df1['Sex']==0),'Age']=mean_men

In [ ]:
df1.isnull().sum()

**Feature Scaling**

In [ ]:
df1.Age=(df1.Age-min(df1.Age))/(max(df1.Age)-min(df1.Age))
df1.Fare=(df1.Fare-min(df1.Fare))/(max(df1.Fare)-min(df1.Fare))

In [ ]:
df1.head()

**Data Modelling**

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(df1.drop(['Survived'],axis=1),df1.Survived,test_size=0.25,random_state=0,stratify=df1.Survived)

**Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression
clf= LogisticRegression()
clf.fit(x_train,y_train)

In [ ]:
from sklearn.metrics import accuracy_score
y_pred= clf.predict(x_test)
accuracy_score (y_test,y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix
mat=confusion_matrix(y_test,y_pred)
mat

In [ ]:
sns.heatmap(mat,annot=True)

**Cleaning the test dataset**

In [ ]:
test.head()

In [ ]:
df2=test.drop(['Name','Ticket','PassengerId','Cabin','Embarked'],axis=1)
df2.head()

In [ ]:
df2['Sex']=df2.Sex.map({'male':0,'female':1})

In [ ]:
df2.head()

In [ ]:
df2.isnull().sum()

In [ ]:
Mean_fem=df2[df2['Sex']==1]['Age'].mean()
Mean_men=df2[df2['Sex']==0]['Age'].mean()

In [ ]:
df2.loc[(df2.Age.isnull())&(df2['Sex']==1),'Age']=Mean_fem
df2.loc[(df2.Age.isnull())&(df2['Sex']==0),'Age']=Mean_men

In [ ]:
df2.isnull().sum()

In [ ]:
df2[df2.Title.isnull()]

In [ ]:
df2['Title']=df2.Title.fillna(3)  #filling with value corresponding to Mrs since age is 39

In [ ]:
df2.isnull().sum()

In [ ]:
df2['Fare']=df2.Fare.fillna(df2.Fare.median())

In [ ]:
df2.isnull().sum()

In [ ]:
df2.head()

In [ ]:
df2['Age']=(df2.Age-min(df2.Age))/(max(df2.Age)-min(df2.Age))

In [ ]:
df2['Fare']=(df2.Fare-min(df2.Fare))/(max(df2.Fare)-min(df2.Fare))

In [ ]:
df2.head()

**Prediction**

In [ ]:
pred=clf.predict(df2)

In [ ]:
pred

In [ ]:
submit=pd.DataFrame({'PassengerId':test['PassengerId'],'Survived':pred})
submit.to_csv('submit.csv',index=False)

In [ ]:
pred_df=pd.read_csv('submit.csv')

In [ ]:
sns.countplot(x='Survived',hue='Survived',data=pred_df)